### This Jupyter notebook will be used to determine the user defines variables

#### Step 1 - Retrieve Location of Desired Study Area
Using either a shapefile, coordinate, or WKT the study area for where satalite imagery will be downloaded is to be determined

In [22]:
import download
from download import *

### Find data using a geographic search
To begin, we will need to craft a suitable WKT. ASF's [Vertex](https://search.asf.alaska.edu) can be helpful in this regard, as it allows you to draw on a map, or import a geospatial file such as a shapefile or geojson, after which a WKT string can be copied and used elsewhere. <br><br>
This method is good for bulk downloads of all scenes matching the description, excercise with caution as it will download the max limit (currently set at 10) and can take awhile.

In [2]:
#Select location using a WKT obtained from ASF's Vertex search site or copied from a geodatabase
#example: POLYGON((-61.0632 -20.9625,-59.9217 -20.9625,-59.9217 -20.425,-61.0632 -20.425,-61.0632 -20.9625))
wktlocation = download.location()

The given location is: POLYGON((-61.0632 -20.9625,-59.9217 -20.9625,-59.9217 -20.425,-61.0632 -20.425,-61.0632 -20.9625))


In [3]:
#set start date for search
dateStart = download.startDate()

Start Date: 2020-01-01
Does start date match format? : True


In [4]:
#set end date for search
dateEnd = download.endDate()

End Date: 2020-01-10
Does end date match format? : True


### Finding data using an existing scene name
If you already have the name of the scene (i.e. 'S1A_EW_GRDM_1SDH_20150205T122016_20150205T122116_004488_005812_F6B0') you can use this part of the program. This is the recommended method as it allows the user to visually verify the scene before downloading.

In [23]:
#create blank list OR run to reset current list
scene_list = []
print(scene_list)

[]


In [24]:
#add scenes to the list to search using ASF's Vertex online app, run as many times as needed to append the list
#to follow the example please input 'S1B_IW_GRDH_1SDV_20200607T223317_20200607T223346_021933_0299FF_1064'
scene_list.append(input('input the scene here (i.e. \'S1B_IW_GRDH_1SDV_20200607T223317_20200607T223346_021933_0299FF_1064'))
print(scene_list)

['S1B_IW_GRDH_1SDV_20200607T223317_20200607T223346_021933_0299FF_1064']


#### Choosing type of download

In [25]:
#choose type of product search
resultChoice=input('Type either: centroid, geo, or scene depending on the search criteria used')

if resultChoice == 'centroid':
    results = download.centroid_loc(wktlocation, dateStart, dateEnd)
elif resultChoice == 'geo':
    results = download.geo_loc(wktlocation, dateStart, dateEnd)
elif resultChoice == 'scene':
    results = download.scene_loc(scene_list)
else:
    print('Error, please try again')

2 results found


#### Login to downlaod data

You to have an [Earthdata Login](https://urs.earthdata.nasa.gov/) account to access the download function. The easiest way to check that your EDL account is in order is to simply go to [Vertex](https://search.asf.alaska.edu) and try to download a product.

In [26]:
#login to ASF Search with Earthdata account
user_session = download.ASF_login()

Success!


#### Download the data
The scene search will download the specific file the user found in [Vertex](https://search.asf.alaska.edu), this may be prefered to geographic search.<br><br>
The geographic search methods are good for bulk downloads of all scenes matching the description, excercise with caution as it will download the max limit (currently set at 10) and can take awhile.

In [27]:
#set the downlaod directory, or create it if it doesnt exist
download.dir_create()

Downlaods can take awhile, with files capable of being mulitple GB each, please be patient

In [29]:
download.download_products(results, user_session)

c:\Users\Kyle\OneDrive\OneDriveDocuments\Coding\ice-cover-change\venv-ice\lib\site-packages\asf_search\download\download.py:61: UserWarning: File already exists, skipping download: ./downloads\S1B_IW_GRDH_1SDV_20200607T223317_20200607T223346_021933_0299FF_1064.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
c:\Users\Kyle\OneDrive\OneDriveDocuments\Coding\ice-cover-change\venv-ice\lib\site-packages\asf_search\download\download.py:61: UserWarning: File already exists, skipping download: ./downloads\S1B_IW_GRDH_1SDV_20200607T223317_20200607T223346_021933_0299FF_1064.iso.xml
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')


### Data Manipulation
I have provided geotiffs in the data folder already to save the trouble of downloading and georeferencing. If self downloaded use following cell to ensure georeferencing is done to the same projection as shapefile

#### Unzip

In [ ]:
#unzip downloaded products
import zipfile
from pathlib import Path

p = Path('.')

for f in p.glob('downloads/*.zip'):
    with zipfile.ZipFile(f, 'r') as archive:
        archive.extractall(path=f'./downloads/{f.stem}')
        print(f'Done {f.stem}')

### Raster Manipulation

In [ ]:
#clip raster images with polygon for study site
import fiona
import rasterio
import rasterio.mask
from rasterio import plot
from rasterio.plot import show

crs = rasterio.crs.CRS({"init": "epsg:4326"})

with fiona.open("data\location_polygon.shp", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open('data\s1b-iw-grd-vv-20200607t223317-20200607t223346-021933-0299ff-001_gr.tiff') as src:
    #src.crs = crs #could not figure this one out... have to open the tif is QGIS and export it with the proper CRS
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("data/clipped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
ds = rasterio.open('data/clipped.tif')
data = ds.read()

max = data.max()
mean = data.mean()

In [ ]:
#reclassification
import numpy as np

lista = data.copy()

lista[np.where((lista >= 0) & (lista <= mean/2))] = 1 #deforested
lista[np.where((lista >= mean/2) & (lista <= mean))] = 2 #meh
lista[np.where((lista >= mean) & (lista <= max))] = 3 #forest

with rasterio.open('data/reclass.tif', 'w', 
driver=ds.driver,
height=ds.height,
width=ds.width,
count=ds.count,
transform=ds.transform,
dtype=data.dtype) as dst:
    dst.write(lista)

In [ ]:
#resample
import rasterio
from rasterio.enums import Resampling

with rasterio.open('data/clipped.tif') as src:
    data = src.read(
        out_shape=(
            src.count,
            int(src.height * 1/2),
            int(src.width * 1/2)
        ),
        resampling=Resampling.bilinear)

# scale image transform
    transform = src.transform * src.transform.scale(
        (src.width / data.shape[-1]),
        (src.height / data.shape[-2])
    )

In [ ]:
import matplotlib
from matplotlib import pyplot

src = rasterio.open("data/reclass.tif")
fig, ax = pyplot.subplots(1, figsize=(12, 12))
show((src, 1), cmap='Greys_r', interpolation='none', ax=ax)
matplotlib.axes._subplots.AxesSubplot
show((src, 1), contour=True, ax=ax)
matplotlib.axes._subplots.AxesSubplot
pyplot.show()

In [ ]:
img2020=rasterio.open('data/clipped20200607.tif')
img2021=rasterio.open('data/clipped20210614.tif')
img=rasterio.open('data/clipped.tif')

### And now the files are ready for maipulation and analysis :D